# Uniswap Helper Functions (Calculations)

In [1]:
import math
q96 = 2 ** 96

## Square root price x96

In [2]:
def price_to_sqrtPrice(price_of_token0, price_of_token1, decimals_of_token0, decimals_of_token1):
    adj_p = (price_of_token1 / price_of_token0) * (10 ** decimals_of_token1 / 10 ** decimals_of_token0)
    return int(math.sqrt(adj_p) * q96)

price_to_sqrtPrice(2000,1,6,18)

1771595571142957166518320255467520

In [4]:
def sqrtPrice_to_price(sqrtPriceX96, decimals_of_token0, decimals_of_token1):
  return ((sqrtPriceX96/q96) ** 2) * (10 ** decimals_of_token0 / 10 ** decimals_of_token1)

p = sqrtPrice_to_price(1771595571142957166518320255467520, 6, 18)
print(p)

0.0005


## Tick Calculation

In [33]:
def tick_to_price(tick, decimals_of_token0, decimals_of_token1):
    price = math.pow(1.0001, tick)
    price_in_terms_of_token0 = price / (10 ** decimals_of_token1 / 10 ** decimals_of_token0)
    price_in_terms_of_token1 = 1 / price_in_terms_of_token0
    return (price_in_terms_of_token0, price_in_terms_of_token1)

tick = 200311
(price_in_terms_of_token0, price_in_terms_of_token1) = tick_to_price(tick, 6, 18)
print(price_in_terms_of_token0, price_in_terms_of_token1)

0.0004999899277897779 2000.0402896525002


In [9]:
def price_to_tick(price_of_token0, price_of_token1, decimals_of_token0, decimals_of_token1):
    p = price_of_token1 / price_of_token0
    adj_p = p * (10 ** decimals_of_token1/ 10 ** decimals_of_token0)
    return math.floor(math.log(adj_p, 1.0001))

c = price_to_tick(2000, 1, 6, 18)
print(c)

200311


## Liquidity Calculations

In [47]:
def calculate_liquidity_at_a_position(amount0, amount1, price, pa, pb):
    liquidity_x = (amount0 * math.sqrt(price) * math.sqrt(pb))/(math.sqrt(pb) - math.sqrt(price))
    liquidity_y = amount1 /(math.sqrt(price) - math.sqrt(pa))
    return(liquidity_x, liquidity_y, int(liquidity_x * q96), int(liquidity_y * q96))

print(calculate_liquidity_at_a_position(2000, 0.78, 1/2000, 1/2500, 1/1500))

(333.8047720482799, 330.41330224498364, 26446738727878082909756031238144, 26178038807140305553825906819072)


## Amount Calculations

### Amount calculation from price range

use this function when you want to calculate amount1 in ration to amount0 you are providing.
<b> Remember price is calculated in terms of x </b>

Example: 
Standard Conversion: 1 ETH (y, token1) = 2000 USDC (x, token0)
P (price) = y/x => 1 / 2000 = 0.0005 ETH (price of y in terms of x)

if my price is 0.0005 ETH, I can set my upper price (P<sub>b</sub>) to be 0.00066 ETH (1/1500) and the lower price (P<sub>a</sub>) to be 0.0004 ETH (1/2500).

(in an inverse rel where x = ETH & y = USDC, (P<sub>b</sub>) = 2500 and (P<sub>a</sub>) = 1500)

if i supply 2000 USDC in this price range how many ETH will i need at the current price?


In [41]:
def amount1FromAmount0(price, pa, pb, amount0):
    liquidity_x = (amount0 * math.sqrt(price) * math.sqrt(pb))/(math.sqrt(pb) - math.sqrt(price))
    amount1 = liquidity_x * (math.sqrt(price) - math.sqrt(pa))
    return amount1

amount1 = amount1FromAmount0(1/2000, 1/2500, 1/1500, 2000); # pass fractions (passing decimals looses precision)
print(amount1, "for 200 USDC")

# if you want to calculate amount1 from amount0 just flip some inputs
amount0 = amount1FromAmount0(2000, 1500, 2500, 1)
print(amount0, "for 1 ETH")

333.8047720482799
0.7880061741721576 for 200 USDC
423.60679774997914
2538.051179739941 for 1 ETH


### Range from amounts of assets

Example: A user has y = 1 ETH and x = 2000 USDC, and wants to use pb = 0.0004 ETH per USDC as the top of the price range. What is the bottom of the range (P<sub>a</sub>) that ensures the opened position uses the full amount of their funds? Find P<sub>b</sub>

In [39]:
def lower_range_from_amount0(price_of_y, price_of_x, upper_range):
    current_price = price_of_y/price_of_x;
    temp = (price_of_y / (math.sqrt(upper_range) * price_of_x)) + math.sqrt(current_price) - (price_of_y / (math.sqrt(current_price) * price_of_x))
    return math.pow(temp, 2)

print(lower_range_from_amount0(1, 2000, 0.0004))

0.0006250000000000002


### Assets after a price change


(333.8047720482799, 330.41330224498364, 26446738727878082909756031238144, 26178038807140305553825906819072)
